## 11. Flaky tests: causes and cures

A **flaky** test sometimes passes, sometimes fails with no code change. Root causes:
* Hidden time assumptions (`sleep(0.1)`) → use explicit waits or free‐running clocks.
* Randomness not seeded (`random.random()`) → set `PYTHONHASHSEED`, seed RNG.
* External services / network latency → mock or retry.
* Order dependence – tests mutate global state.

Detection: run with `pytest -n auto --lf --maxfail=1 --reruns 3` in CI; quarantine & fix quickly.

```python
import random, pytest
def lucky():
    return random.randint(0,1)
def test_lucky():
    random.seed(0)
    assert lucky() == 1  # deterministic now
```

### Quick check

1. Race conditions often manifest as:
  a. deterministic failures  b. intermittent failures

2. True / False Setting PYTHONHASHSEED fixes order‐dependent dictionaries.

<details><summary>Answer key</summary>

1. **b**.
2. **True** – forces repeatable hash order.

</details>

## 12. Code coverage with `coverage.py` & `pytest-cov`

`coverage run -m pytest` or `pytest --cov=src --cov-report=term-missing` measures which lines executed.
* **Line coverage** – percentage of lines touched.
* **Branch coverage** – covers if/else arms.
Aim for risk‑based target (≈80%), but value lies in **what remains untested**.

```bash
pytest --cov=project --cov-report=html
# opens htmlcov/index.html with colourised gaps
```

### Quick check

1. Missing branch coverage might hide:
  a. dead code  b. well‑tested paths only

2. True / False 100 % coverage guarantees bug‑free code.

<details><summary>Answer key</summary>

1. **a**.
2. **False** – tests could assert nothing.

</details>

## 13. Behaviour‑driven hints (Given‑When‑Then)

BDD frames tests as **specifications** in business language:
* **Given** context
* **When** action
* **Then** expected outcome

Tools: `pytest-bdd`, `behave`. Keeps stakeholders engaged and clarifies intent.

```python
# test_login.feature
Scenario: Successful login
  Given I am on the login page
  When I submit valid credentials
  Then I should see the dashboard
```

### Quick check

1. BDD primarily targets:
  a. developers only  b. dev + business

2. True / False pytest can integrate BDD via plugins.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>

## 14. Property‑based testing with Hypothesis

Instead of hand‑picked examples, **Hypothesis** generates random inputs satisfying strategies and tries to falsify invariants. Finds edge cases humans miss.
Key API: `@given(ints(), lists(ints()))` and `assume` / `reject`.

```python
from hypothesis import given, strategies as st
@given(st.lists(st.integers()))
def test_rev_rev(xs):
    assert list(reversed(list(reversed(xs)))) == xs
```

### Quick check

1. Hypothesis shrinks failing data to:
  a. largest case  b. minimal counterexample

2. True / False Property tests can replace all unit tests.

<details><summary>Answer key</summary>

1. **b**.
2. **False** – complement, not replace.

</details>

## 15. Performance benchmarks & profiling

`pytest-benchmark` measures wall‑time and compares to saved baseline. For deeper insight use `cProfile`, `snakeviz`, or `py-spy` flamegraphs.
Micro‑benchmarks must run on quiet CPU; pin process with taskset or use CI perf machines.

```python
# test_speed.py
def fib(n):
    a,b=0,1
    for _ in range(n): a,b=b,a+b
    return a
def test_bench(benchmark):
    benchmark(fib, 1000)
```

### Quick check

1. `pytest-benchmark` fails build if slower than baseline when flag:
  a. --benchmark-autosave  b. --benchmark-fail=0%


2. True / False `py-spy` injects without modifying source code.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>